In [1]:
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torchvision
from torchvision import transforms, utils

from create_dataset import ArtistsPaintingsDataset
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from NeuralNet import Net

from sklearn.model_selection import cross_val_predict


In [6]:
transform = transforms.Compose([transforms.CenterCrop(size=(256,256)),
                                transforms.ToTensor()])

# actual artists
# artists_idx = [8,13,15,16,19,20,22,30,31,32,46]

# debug list
artists_idx = [1,2]

train_data = ArtistsPaintingsDataset(transform = transform, mode="Train", artists_idx=artists_idx)
test_data = ArtistsPaintingsDataset(transform = transform, mode="Test", artists_idx=artists_idx)
print(len(train_data), "train images loaded.")
print(len(test_data), "test images loaded.")

train_loader = DataLoader(train_data, batch_size=256)
test_loader = DataLoader(test_data)

100%|██████████| 16/16 [00:00<00:00, 61.83it/s]

142 train images loaded.
16 test images loaded.


In [ ]:
def validate_lr(X,y,lr,train_loader):
    net = Net(size= (256,256), num_classes = 11)
    loss = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr = lr, momentum = 0.9) 
    for epoch in range(5):
        print(epoch)
        for batch in tqdm(train_loader):
            x_train, y_train = batch[0]
            optimizer.zero_grad()
            net.train()
            output = net(x_train)
            training_loss = loss(output, y_train)
            training_loss.backward()
            optimizer.step()
    scores = cross_val_score(net, X, y, cv=5)
    return scores

In [7]:
x_cv, y_cv = train_data.samples, train_data.targets
x_cv = torch.cat(x_cv, dim = 0).resize(len(y_cv),3,256,256)
print(x_cv.shape)
y_cv = torch.tensor(y_cv)
print(y_cv.shape)

torch.Size([142, 3, 256, 256])
torch.Size([142])


In [17]:
X = {
    'key_a': np.random.random((1000, 10)).astype(np.float32),
    'key_b': np.random.random((1000, 20)).astype(np.float32),
}
y = np.random.randint(0, 2, size=1000)

from skorch import NeuralNetClassifier
net = NeuralNetClassifier(Net(size= (256,256), num_classes = 2),
                         train_split=None,
                         criterion = torch.nn.CrossEntropyLoss(),
                         optimizer = torch.optim.SGD,
                         lr = 0.1,
                         optimizer__momentum = 0.9,
                         dataset = train_data,
                         batch_size = 256)
scores = cross_val_predict(net, X, y, cv=3)

ValueError: Found input variables with inconsistent numbers of samples: [2, 1000]